In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
from utils import *
import pandas as pd

Output()

In [26]:
df = pd.read_csv('../data/cl_papers.csv', index_col=0)
df

,id,title,area,source,year,doi,abstract,semantic_scholar_id
0,main.1004,AnswerFact: Fact Checking in Product Question ...,Question Answering,EMNLP,2020,10.18653/v1/2020.emnlp-main.188,Product-related question answering platforms n...,4c61df1b4b9a164fec1a34587b4fffae029cd18c
1,main.1006,Knowledge-Grounded Dialogue Generation with Pr...,Dialog and Interactive Systems,EMNLP,2020,10.18653/v1/2020.emnlp-main.272,We study knowledge-grounded dialogue generatio...,3447a432f724aa36595643446acda5b78943db19
2,main.1009,BiST: Bi-directional Spatio-Temporal Reasoning...,Dialog and Interactive Systems,EMNLP,2020,10.18653/v1/2020.emnlp-main.145,Video-grounded dialogues are very challenging ...,f4a2acfeb1705df3f430cc53ace26e1dbbbcbd16
3,main.1010,A Knowledge-Aware Sequence-to-Tree Network for...,NLP Applications,EMNLP,2020,10.18653/v1/2020.emnlp-main.579,With the advancements in natural language proc...,24ed85ad966823868c1694a19385d01c6ad71008
4,main.1011,Knowledge Association with Hyperbolic Knowledg...,Information Extraction,EMNLP,2020,10.18653/v1/2020.emnlp-main.460,Capturing associations for knowledge graphs (K...,3d61a28b9429fc8f7047fc379a0134a3765edbcb
...,...,...,...,...,...,...,...,...
9250,T4773,Rank-Aware Negative Training for Semi-Supervis...,Machine Learning for NLP,ACL,2023,10.1162/tacl_a_00574,Abstract Semi-supervised text classification-b...,79a502caa0b12573f56a7e8948459722aa891479
9251,T4777,Transparency Helps Reveal When Language Models...,"Linguistic Theories, Cognitive Modeling, and P...",ACL,2023,10.1162/tacl_a_00565,Many current NLP systems are built from langua...,eee70790ced38b0fcf50351dbb3dcbdb582467e8
9252,T4803,Design Choices for Crowdsourcing Implicit Disc...,Discourse and Pragmatics,ACL,2023,10.1162/tacl_a_00586,Abstract Disagreement in natural language anno...,bf65a1b637e0f253abb8402fc9d0ab80db79aa92
9253,T4929,Time-and-Space-Efficient Weighted Deduction,"Semantics: Sentence-level Semantics, Textual I...",ACL,2023,10.1162/tacl_a_00588,Abstract Many NLP algorithms have been describ...,f1803d124e5af7263ce656f7ccf3219eacc62b1c


In [4]:
import requests
import bs4

def get_abstract_with_acl_anthology(doi):
    url_path = doi.split('/')[-1]
    url = 'https://aclanthology.org/' + url_path
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    css_selector = "#main > div.row.acl-paper-details > div.col.col-lg-10.order-2 > div > div > span"
    element = soup.select_one(css_selector)
    abstract = element.text
    return abstract

def get_abstract_from_semantic_scholar(paper_id):
    query = f"https://api.semanticscholar.org/graph/v1/paper/{paper_id}"
    fields = "abstract"
    response = requests.get(query, headers={"x-api-key": API_KEY}, params={"fields": fields})
    paper_dict = response.json()

    assert type(paper_dict['abstract']) == str
    return paper_dict['abstract']


def get_abstract_from_crossref(doi):
    url = 'https://api.crossref.org/works/' + doi
    r = requests.get(url)
    response = r.json()
    soup = bs4.BeautifulSoup(response['message']['abstract'], 'html.parser')
    return soup.find('jats:p').text



In [8]:
def get_abstract(row):
    doi = row['doi']
    semantic_scholar_id = row['semantic_scholar_id']

    try:
        if not pd.isna(semantic_scholar_id):
            abstract = get_abstract_from_semantic_scholar(semantic_scholar_id)
            return abstract
    except Exception as e:
        print('failed to abstract from semantic scholar for', semantic_scholar_id)
        print(e)

    try:
        if not pd.isna(doi):
            abstract = get_abstract_with_acl_anthology(doi)
            return abstract
    except Exception as e:
        print('failed to abstract from acl anthology for', doi)
        print(e)

    try:
        if not pd.isna(doi):
            abstract = get_abstract_from_crossref(doi)
            return abstract
    except Exception as e:
        print('failed to abstract from semantic scholar for', doi)
        print(e)

    print('no source has the abstract for', doi)
    return None


In [33]:
### some papers did not have their abstract by any of the sources
### these abstracts where added manually

title_to_doi = {
"ProtAugment: Intent Detection Meta-Learning through Unsupervised Diverse Paraphrasing": '10.18653/v1/2021.acl-long.191',
"Multi-Source Multi-Type Knowledge Exploration and Exploitation for Dialogue Generation": '10.18653/v1/2023.emnlp-main.771',
"DUMB: A Dutch Model Benchmark": '10.18653/v1/2023.emnlp-main.447',
"`Don't Get Too Technical with Me': A Discourse Structure-Based Framework for Automatic Science Journalism": '10.18653/v1/2023.emnlp-main.76',
"A Framework for Vision-Language Warm-up Tasks in Multimodal Dialogue Models": '10.18653/v1/2023.emnlp-main.167',
"Exploring Linguistic Probes for Morphological Inflection": '10.18653/v1/2023.emnlp-main.552',
"It Ain't Over: A Multi-aspect Diverse Math Word Problem Dataset": '10.18653/v1/2023.emnlp-main.927',
"PhenotypeCLIP: Phenotype-based Contrastive Learning for Medical Imaging Report Generation": '10.18653/v1/2023.emnlp-main.989',
"Improving Author Attribute Prediction by Retrofitting Linguistic Representations with Homophily": '10.18653/v1/D18-1070',
"Identifying the narrative styles of YouTube’s vloggers": '10.18653/v1/D18-1394',
"Card-660: A Reliable Evaluation Framework for Rare Word Representation Models": '10.18653/v1/D18-1169',
"Personalized Microblog Sentiment Classification via Adversarial Cross-lingual learning": '10.18653/v1/D18-1031',
"Bootstrapping Transliteration with Guided Discovery for Low-Resource Languages": '10.18653/v1/D18-1046',
"XNLI: Cross-lingual Sentence Understanding through Inference": '10.18653/v1/D18-1269',
}

for title, doi in title_to_doi.items():
    print(title)
    assert len(df[df['title'] == title]) == 1
    df.loc[df['title'] == title, 'doi'] = doi
    

ProtAugment: Intent Detection Meta-Learning through Unsupervised Diverse Paraphrasing
Multi-Source Multi-Type Knowledge Exploration and Exploitation for Dialogue Generation
DUMB: A Dutch Model Benchmark
`Don't Get Too Technical with Me': A Discourse Structure-Based Framework for Automatic Science Journalism
A Framework for Vision-Language Warm-up Tasks in Multimodal Dialogue Models
Exploring Linguistic Probes for Morphological Inflection
It Ain't Over: A Multi-aspect Diverse Math Word Problem Dataset
PhenotypeCLIP: Phenotype-based Contrastive Learning for Medical Imaging Report Generation
Improving Author Attribute Prediction by Retrofitting Linguistic Representations with Homophily
Identifying the narrative styles of YouTube’s vloggers
Card-660: A Reliable Evaluation Framework for Rare Word Representation Models
Personalized Microblog Sentiment Classification via Adversarial Cross-lingual learning
Bootstrapping Transliteration with Guided Discovery for Low-Resource Languages
XNLI: Cro

In [34]:
from tqdm import tqdm

for i in tqdm(range(len(df))):
    if pd.isna(df.iloc[i]['abstract']):
        abstract = get_abstract(df.iloc[i])
        df.iloc[i, df.columns.get_loc('abstract')] = abstract


 56%|█████████████████████████████████████████████████████████████████████████████████████████▉                                                                      | 5205/9255 [00:00<00:00, 52044.71it/s]

failed to abstract from semantic scholar for c981f606740220fcac476a552d9353a825ea4aa9

failed to abstract from semantic scholar for d240ea959f383411983a536089aeeab6bdf0163d

failed to abstract from semantic scholar for 74fe9a7742c362e4235af69fab75c7f0e381a5e0

failed to abstract from semantic scholar for 8106093c872e3818bfcd9001ed1f0a0ae1965e02

failed to abstract from semantic scholar for 7b95148986fe48e09cfe80df5518e466a42d4e4c

failed to abstract from semantic scholar for 3759c5853d4b26c011d70c0bb290a00e1015a610



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9255/9255 [00:11<00:00, 837.55it/s]


In [36]:
df.to_csv('../data/cl_papers.csv')

In [35]:
df[df['abstract'].isna()]

,id,title,area,source,year,doi,abstract,semantic_scholar_id


# Computing the embeddings

In [34]:
from transformers import AutoTokenizer
from adapters import AutoAdapterModel
from tqdm import tqdm
tqdm.pandas()

tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')

model = AutoAdapterModel.from_pretrained("allenai/specter2_base")
adapter_name = model.load_adapter("allenai/specter2_classification", source="hf", set_active=True)

def get_embedding(paper_row):
    text = paper_row['title'] + tokenizer.sep_token + paper_row['abstract']
    inputs = tokenizer(text,
                       padding=True,
                       truncation=True,
                       return_tensors="pt",
                       return_token_type_ids=False,
                       max_length=2048)
    output = model(**inputs)
    embeddings = output.last_hidden_state[:, 0, :][0].detach().numpy()
    return embeddings

processed_df['embedding'] = processed_df.progress_apply(get_embedding, axis=1)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9282/9282 [14:28<00:00, 10.69it/s]


In [35]:
processed_df

,id,title,area,interpretability,doi,source,working_doi,abstract,embedding
0,main.8,Large Scale Multi-Actor Generative Dialog Mode...,Dialogue and Interactive Systems,False,10.18653/v1/2020.acl-main.8,ACL2020,True,Non-goal oriented dialog agents (i.e. chatbots...,"[-0.55811894, -0.12536883, -0.06339799, -1.815..."
1,main.52,CDL: Curriculum Dual Learning for Emotion-Cont...,Dialogue and Interactive Systems,False,10.18653/v1/2020.acl-main.52,ACL2020,True,Emotion-controllable response generation is an...,"[-1.1278496, -0.5229794, 0.0056311972, -1.3223..."
2,main.46,Emergence of Syntax Needs Minimal Supervision,Theory and Formalism in NLP (Linguistic and Ma...,False,10.18653/v1/2020.acl-main.46,ACL2020,True,This paper is a theoretical contribution to th...,"[0.26176804, 0.8106163, 0.27426642, -1.174295,..."
3,main.359,Selecting Backtranslated Data from Multiple So...,Machine Translation,False,10.18653/v1/2020.acl-main.359,ACL2020,True,Machine translation (MT) has benefited from us...,"[-0.43927717, 1.0674063, 0.08589529, -0.437168..."
4,main.417,ParaCrawl: Web-Scale Acquisition of Parallel C...,Resources and Evaluation,False,10.18653/v1/2020.acl-main.417,ACL2020,True,We report on methods to create the largest pub...,"[-0.3593886, 0.33652788, -0.026537634, -0.7595..."
...,...,...,...,...,...,...,...,...,...
9277,889,Multimodal Transformer for Unaligned Multimoda...,"Vision, Robotics, Multimodal, Grounding and Sp...",NaN,10.18653/v1/P19-1656,ACL2019,True,"Human language is often multimodal, which comp...","[-0.25314885, -0.08230631, -0.3357741, -1.7636..."
9278,2155,"Show, Describe and Conclude: On Exploiting the...","Vision, Robotics, Multimodal, Grounding and Sp...",NaN,10.18653/v1/P19-1657,ACL2019,True,Chest X-Ray (CXR) images are commonly used for...,"[-0.488766, -0.30718938, -1.3068513, -0.397652..."
9279,384,Visual Story Post-Editing,"Vision, Robotics, Multimodal, Grounding and Sp...",NaN,10.18653/v1/P19-1658,ACL2019,True,We introduce the first dataset for human edits...,"[-0.63484645, -0.21893704, 0.09538727, -0.8349..."
9280,1891,Multimodal Abstractive Summarization for How2 ...,"Vision, Robotics, Multimodal, Grounding and Sp...",NaN,10.18653/v1/P19-1659,ACL2019,True,"In this paper, we study abstractive summarizat...","[-0.9391148, -0.115553305, -0.06912733, -1.159..."


In [37]:
processed_df.to_csv('../data/new_classifier_data.csv')